# 02 — LangGraph Basics

In this notebook, you’ll learn the core ideas behind **LangGraph**, a modern orchestration library built for **agentic workflows**.

LangGraph lets you represent your logic as a **graph** — where each node is a runnable component (LLM, function, tool, etc.), and edges define how data flows between them.

By the end of this notebook, you'll understand how to:
- Create a minimal LangGraph graph
- Define nodes and transitions
- Pass and update **state** across nodes
- Execute the graph step by step

In [1]:
# --- Environment setup (load API keys & optional model) ---
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # Load .env if present

openai_api_key = os.getenv("OPENAI_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

print("✅ Environment loaded.")

# Optional: uncomment if you want to test with an actual model
# from langchain_openai import ChatOpenAI
# chat_model = ChatOpenAI(model="gpt-4o-mini")

✅ Environment loaded.


## 🧩 What is LangGraph?

LangGraph is a library from the LangChain team that enables **graph-based orchestration**.

Instead of chaining components linearly (prompt → model → parser), LangGraph allows you to build **directed graphs** of computation — each node can represent a component or an agent.

### Why graphs?
- **Visual clarity**: you can visualize logic as a flowchart.
- **Stateful control**: maintain and mutate state between nodes.
- **Branching logic**: run different paths depending on results.
- **Reusability**: each node can be reused independently.

## ⚙️ Core Concepts

| Concept | Description |
|----------|--------------|
| **Graph** | The full system — a collection of connected nodes. |
| **Node** | A single step in the process (LLM, function, or chain). |
| **Edge** | The transition that connects nodes — defines the flow. |
| **State** | The shared data dictionary that moves and updates through the graph. |
| **Checkpointer** | Optional — saves and restores state across runs (for memory). |

## 🚀 Example 1 — Minimal LangGraph Graph

Let’s build a minimal graph with two nodes:
1. **Start** → prints an initial message.
2. **End** → transforms the input and stops.

This shows the basic structure of a LangGraph pipeline.

In [2]:
from langgraph.graph import StateGraph, START, END

# 1️⃣ Define the state (shared data type)
def add_message(state, new_text):
    return {"messages": state.get("messages", []) + [new_text]}

# 2️⃣ Define node functions
def start_node(state):
    print("Start node running...")
    return add_message(state, "Graph started successfully!")

def end_node(state):
    print("End node reached.")
    return add_message(state, "Graph execution complete.")

# 3️⃣ Build the graph
graph = StateGraph(dict)
graph.add_node("start", start_node)
graph.add_node("end", end_node)
graph.add_edge(START, "start")
graph.add_edge("start", "end")
graph.add_edge("end", END)

graph_instance = graph.compile()

# 4️⃣ Run the graph
output = graph_instance.invoke({"messages": []})
print("\n✅ Final state:", output)

Start node running...
End node reached.

✅ Final state: {'messages': ['Graph started successfully!', 'Graph execution complete.']}


## 🔍 What happens step by step

1. **START** → triggers `start_node()`
2. `start_node()` adds a new message to the state.
3. Control flows to `end_node()`
4. `end_node()` modifies the state again.
5. The graph ends, returning the updated `state` dictionary.

💡 Output example:
```python
{'messages': ['Graph started successfully!', 'Graph execution complete.']}
```

## 🧠 Why this matters

Even this minimal example already shows LangGraph’s power:
- You have **clear nodes and edges** (no hidden control flow).
- You can inspect and manipulate **state** easily.
- You can later replace `print()` nodes with **LLM calls**, **retrievers**, **tools**, or **memory checkpointers**.

That’s exactly what we’ll do in the next notebooks.

## ✅ Summary

- **LangGraph** organizes logic as a graph (nodes, edges, state).
- Each node is a **runnable** (function, model, or chain).
- The graph manages execution order and shared state.
- You can easily extend this to include **memory** and **LLMs**.